In [1]:
import pandas as pd
import numpy as np
from collections import OrderedDict

pd.set_option('display.max_columns',None)
pd.set_option('display.max_rows',None)
pd.set_option('display.max_info_columns',1000)

In [2]:
application_test = pd.read_csv('../application_test.csv')
application_train  = pd.read_csv('../application_train.csv')
bureau = pd.read_csv('../bureau.csv')
bureau_balance = pd.read_csv('../bureau_balance.csv')
credit_card_balance = pd.read_csv('../credit_card_balance.csv')
installments_payments = pd.read_csv('../installments_payments.csv')
POS_CASH_balance = pd.read_csv('../POS_CASH_balance.csv')
previous_application = pd.read_csv('../previous_application.csv')

In [3]:
print ('the count of {} is {},approximately {} thousands'.format('application_test',application_test.shape[0],application_test.shape[0]/1000))
print ('the count of {} is {},approximately {} thousands'.format('application_train',application_train.shape[0],application_train.shape[0]/1000))
print ('the count of {} is {},approximately {} thousands'.format('bureau',bureau.shape[0],bureau.shape[0]/1000))
print ('the count of {} is {},approximately {} thousands'.format('bureau_balance',bureau_balance.shape[0],bureau_balance.shape[0]/1000))
print ('the count of {} is {},approximately {} thousands'.format('credit_card_balance',credit_card_balance.shape[0],credit_card_balance.shape[0]/1000))
print ('the count of {} is {},approximately {} thousands'.format('installments_payments',installments_payments.shape[0],installments_payments.shape[0]/1000))
print ('the count of {} is {},approximately {} thousands'.format('POS_CASH_balance',POS_CASH_balance.shape[0],POS_CASH_balance.shape[0]/1000))
print ('the count of {} is {},approximately {} thousands'.format('previous_application',previous_application.shape[0],previous_application.shape[0]/1000))

the count of application_test is 48744,approximately 48.744 thousands
the count of application_train is 307511,approximately 307.511 thousands
the count of bureau is 1716428,approximately 1716.428 thousands
the count of bureau_balance is 27299925,approximately 27299.925 thousands
the count of credit_card_balance is 3840312,approximately 3840.312 thousands
the count of installments_payments is 13605401,approximately 13605.401 thousands
the count of POS_CASH_balance is 10001358,approximately 10001.358 thousands
the count of previous_application is 1670214,approximately 1670.214 thousands


### remain the meta data for train and test

In [4]:
train_target_sequence = application_train[['SK_ID_CURR','TARGET']]
test_sequence = application_test['SK_ID_CURR']
application_train = application_train.drop(['TARGET'], axis=1)

In [5]:
application_train.head()

,SK_ID_CURR,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_TYPE_SUITE,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OWN_CAR_AGE,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,ORGANIZATION_TYPE,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,APARTMENTS_AVG,BASEMENTAREA_AVG,YEARS_BEGINEXPLUATATION_AVG,YEARS_BUILD_AVG,COMMONAREA_AVG,ELEVATORS_AVG,ENTRANCES_AVG,FLOORSMAX_AVG,FLOORSMIN_AVG,LANDAREA_AVG,LIVINGAPARTMENTS_AVG,LIVINGAREA_AVG,NONLIVINGAPARTMENTS_AVG,NONLIVINGAREA_AVG,APARTMENTS_MODE,BASEMENTAREA_MODE,YEARS_BEGINEXPLUATATION_MODE,YEARS_BUILD_MODE,COMMONAREA_MODE,ELEVATORS_MODE,ENTRANCES_MODE,FLOORSMAX_MODE,FLOORSMIN_MODE,LANDAREA_MODE,LIVINGAPARTMENTS_MODE,LIVINGAREA_MODE,NONLIVINGAPARTMENTS_MODE,NONLIVINGAREA_MODE,APARTMENTS_MEDI,BASEMENTAREA_MEDI,YEARS_BEGINEXPLUATATION_MEDI,YEARS_BUILD_MEDI,COMMONAREA_MEDI,ELEVATORS_MEDI,ENTRANCES_MEDI,FLOORSMAX_MEDI,FLOORSMIN_MEDI,LANDAREA_MEDI,LIVINGAPARTMENTS_MEDI,LIVINGAREA_MEDI,NONLIVINGAPARTMENTS_MEDI,NONLIVINGAREA_MEDI,FONDKAPREMONT_MODE,HOUSETYPE_MODE,TOTALAREA_MODE,WALLSMATERIAL_MODE,EMERGENCYSTATE_MODE,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,FLAG_DOCUMENT_2,FLAG_DOCUMENT_3,FLAG_DOCUMENT_4,FLAG_DOCUMENT_5,FLAG_DOCUMENT_6,FLAG_DOCUMENT_7,FLAG_DOCUMENT_8,FLAG_DOCUMENT_9,FLAG_DOCUMENT_10,FLAG_DOCUMENT_11,FLAG_DOCUMENT_12,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100002,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,351000.0,Unaccompanied,Working,Secondary / secondary special,Single / not married,House / apartment,0.018801,-9461,-637,-3648.0,-2120,NaN,1,1,0,1,1,0,Laborers,1.0,2,2,WEDNESDAY,10,0,0,0,0,0,0,Business Entity Type 3,0.083037,0.262949,0.139376,0.0247,0.0369,0.9722,0.6192,0.0143,0.00,0.0690,0.0833,0.1250,0.0369,0.0202,0.0190,0.0000,0.0000,0.0252,0.0383,0.9722,0.6341,0.0144,0.0000,0.0690,0.0833,0.1250,0.0377,0.022,0.0198,0.0,0.0,0.0250,0.0369,0.9722,0.6243,0.0144,0.00,0.0690,0.0833,0.1250,0.0375,0.0205,0.0193,0.0000,0.00,reg oper account,block of flats,0.0149,"Stone, brick",No,2.0,2.0,2.0,2.0,-1134.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
1,100003,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,1129500.0,Family,State servant,Higher education,Married,House / apartment,0.003541,-16765,-1188,-1186.0,-291,NaN,1,1,0,1,1,0,Core staff,2.0,1,1,MONDAY,11,0,0,0,0,0,0,School,0.311267,0.622246,NaN,0.0959,0.0529,0.9851,0.7960,0.0605,0.08,0.0345,0.2917,0.3333,0.0130,0.0773,0.0549,0.0039,0.0098,0.0924,0.0538,0.9851,0.8040,0.0497,0.0806,0.0345,0.2917,0.3333,0.0128,0.079,0.0554,0.0,0.0,0.0968,0.0529,0.9851,0.7987,0.0608,0.08,0.0345,0.2917,0.3333,0.0132,0.0787,0.0558,0.0039,0.01,reg oper account,block of flats,0.0714,Block,No,1.0,0.0,1.0,0.0,-828.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
2,100004,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,135000.0,Unaccompanied,Working,Secondary / secondary special,Single / not married,House / apartment,0.010032,-19046,-225,-4260.0,-2531,26.0,1,1,1,1,1,0,Laborers,1.0,2,2,MONDAY,9,0,0,0,0,0,0,Government,NaN,0.555912,0.729567,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N

### put the test and train together

In [6]:
application_all = application_train.append(application_test)
print ('all dataset counts is {}'.format(application_all.shape[0]))

all dataset counts is 356255


### do some regular processes for the data

1. drop the the columns with more than 30 percent NA values
2. drop the unnecessary columns
3. fillNa by when int/float then mean, when object then mode 

In [7]:
def miss_ratio_dict(df):
    """
    calculate the miss data ratio
    :param df:DataFrame
    :return: a dict which the columns and its miss precentage
    """
    columns_NA_ratio = OrderedDict()
    for columns in df.columns:
        df_na = df[columns].isnull()
        na_ratio = df_na[df_na==True].shape[0]/df_na.shape[0]
        columns_NA_ratio[columns] = round(na_ratio*100,2)
    return columns_NA_ratio

In [8]:
def fillna_values_dict(df):
    """
    get the fillna values for each columns
    if dtype is numeric,then fillna values is mean
    if dtype is object,then fillna value is mode
    :param df:
    :return:dict,k is columns,v is fillna value
    """
    columns_NA_fill = OrderedDict()
    for columns in df.columns:
        dtypes = df[columns].dtype.name
        if 'int' in dtypes or 'float' in dtypes:
            columns_NA_fill[columns] = df[columns].mean()
            continue
        if dtypes == 'object':
            columns_NA_fill[columns] = df[columns].mode()[0]
            continue
    print ('{} is not found dtypes'.format(dtypes))
    return columns_NA_fill

In [9]:
datasets = [bureau,bureau_balance,credit_card_balance,installments_payments,POS_CASH_balance,previous_application]

In [12]:
for ds in datasets:
    miss_ratio = miss_ratio_dict(ds)
    #print (miss_ratio)
    drop_columns = list()
    for column in ds.columns:
        if miss_ratio[column] > 20:
            drop_columns.append(column)
    print (drop_columns)
    #ds.drop(drop_columns, axis=1, inplace=True)     

[]
[]
[]
[]
[]
[]


In [11]:
for ds in datasets:
    fillna_values = fillna_values_dict(ds)
    print (fillna_values)
    ds = ds.fillna(fillna_values)

int64 is not found dtypes
OrderedDict([('SK_ID_CURR', 278214.93364475528), ('SK_ID_BUREAU', 5924434.4890318727), ('CREDIT_ACTIVE', 'Closed'), ('CREDIT_CURRENCY', 'currency 1'), ('DAYS_CREDIT', -1142.1076846800449), ('CREDIT_DAY_OVERDUE', 0.81816656451654246), ('DAYS_CREDIT_ENDDATE', 510.51736230309615), ('CNT_CREDIT_PROLONG', 0.00641040579622332), ('AMT_CREDIT_SUM', 354994.59191767266), ('AMT_CREDIT_SUM_DEBT', 137085.11995216066), ('AMT_CREDIT_SUM_OVERDUE', 37.912757741658837), ('CREDIT_TYPE', 'Consumer credit'), ('DAYS_CREDIT_UPDATE', -593.74831976639859)])
object is not found dtypes
OrderedDict([('SK_ID_BUREAU', 6036297.3329735519), ('MONTHS_BALANCE', -30.741686652985312), ('STATUS', 'C')])
int64 is not found dtypes
OrderedDict([('SK_ID_PREV', 1904503.5898997269), ('SK_ID_CURR', 278324.20728888694), ('MONTHS_BALANCE', -34.521920614783383), ('AMT_BALANCE', 58300.155261857624), ('AMT_CREDIT_LIMIT_ACTUAL', 153807.95740033622), ('AMT_DRAWINGS_ATM_CURRENT', 5961.3248220593059), ('AMT_DRAW

### let's create some feature joining with main table